In [1]:
from common import *
%matplotlib inline
canvas(12, 9)

/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/common.py:701: RuntimeWarning: divide by zero encountered in log
  return func(obj, *args, **kwargs)


In [2]:
from models.xgbf import pack, packname
from models.split import splitf
from helper.xgbhelp import *
import xgboost as xgb

In [3]:
p = O()
list(packname)

[{.simple}, {.goeasy}, {.gohard}, {.stcktm}, {.ctsint}]

In [4]:
s = splitf(dh, 2, ho=0)

In [5]:
# initialization and work code
p.simple = pack.simple(s)
p.ctsint = pack.ctsint(s)
p.goeasy = pack.goeasy(s)
p.stcktm = pack.stcktm(s)

pack.simple{ asis wqtl basic day pca tsne ymean }
pack.ctsint{ asis wqtl basic day pca tsne x3easy x3easywqtl xgeasy xgeasywqtl xowindiff x3windiff xgwindiff xonrmdiff x3nrmdiff xgnrmdiff xonrmplus x3nrmplus xgnrmplus }
pack.goeasy{ asis wqtl basic day pca tsne featenc x3easy x3easywqtl xgeasy xgeasywqtl xowindiff x3windiff xgwindiff xonrmdiff x3nrmdiff xgnrmdiff xonrmplus x3nrmplus xgnrmplus targenc frac }
pack.stcktm{ asis wqtl basic day pca tsne featenc xowindiff xonrmdiff xonrmplus targenc frac }


In [ ]:
boost_hp = dict(objective='reg:linear', tree_method='hist', booster='gbtree', base_score=0,
                max_depth=3, gamma=50, min_child_weight=1000, eta=.1, seed=0)
train_hp = dict(num_boost_round=100000, early_stopping_rounds=100, verbose_eval=100)
bt = BoostTransformer()
thepath = top_dir + 'data/progress/xgboost/ohfuck/'
#
train_hp['num_boost_round'] = 100000
num_boost_round, early_stopping_rounds = train_hp['num_boost_round'], train_hp['early_stopping_rounds']
Yx__linfp = np.load(top_dir + 'data/progress/linfinal/firstpick.npy')
Y_wmean = bt.o.Y_wmean.makeil_like(dh)
bases = [Yx__linfp, Yx__linfp - Y_wmean, dh.Day * 0.0, Y_wmean]
for packname, ibase, transname in itertools.product(
        ['stcktm', 'ctsint'], range(len(bases)), ['id', 'scl', 'std']):
    print(packname, ibase, transname, '=======================================================================', flush=True)
    base = bases[ibase]
    trans = getattr(bt, transname)
    filename = 'split(tc-lb)|base({})*{}{{{}}}'.format(['linFP', 'linFP`1', 0, 1][ibase], transname, packname)
    X = p[packname]
    T, WT = trans.to_T(*bt.boost(base).to_T(dh.y, dh.w))
    dmat = make_dmat(X, T, WT, s, include='tr cv tc lb')
    train_hp['evals'], train_hp['early_stopping_rounds'] = [[dmat.cv, 'cv']], early_stopping_rounds
    bstcv = xgb.train(boost_hp, dmat.tr, **train_hp)
    #
    bstcv.save_model(thepath + 'trcv/' + filename + '.xgboost')
    Tcv = np.concatenate([bstcv.predict(dmat.tc), bstcv.predict(dmat.lb)])
    Ycv = bt.boost(base).to_Y(trans.to_Y(Tcv))
    np.save(thepath + 'trcv/' + filename + '.T.npy', Tcv)
    np.save(thepath + 'trcv/' + filename + '.Y.npy', Ycv)
    #
    np.save(thepath + 'ntree/' + filename + '.ntree.npy', bstcv.best_ntree_limit)
    train_hp['num_boost_round'] = int(bstcv.best_ntree_limit * 1.25)
    del train_hp['evals']; del train_hp['early_stopping_rounds']
    bst = xgb.train(boost_hp, dmat.tc, **train_hp)
    train_hp['num_boost_round'] = num_boost_round
    #
    bst.save_model(thepath + 'tclb/' + filename + '.xgboost')
    Tx = np.concatenate([bst.predict(dmat.tc), bst.predict(dmat.lb)])
    Yx = bt.boost(base).to_Y(trans.to_Y(Tx))
    np.save(thepath + 'tclb/' + filename + '.T.npy', Tx)
    np.save(thepath + 'tclb/' + filename + '.Y.npy', Yx)

stcktm 0 id =======================================================================
[0]	cv-rmse:2.43413
Will train until cv-rmse hasn't improved in 100 rounds.
[100]	cv-rmse:2.42265
Stopping. Best iteration:
[62]	cv-rmse:2.42247

stcktm 0 scl =======================================================================
[0]	cv-rmse:0.735356
Will train until cv-rmse hasn't improved in 100 rounds.
[100]	cv-rmse:0.734776
Stopping. Best iteration:
[18]	cv-rmse:0.733608

stcktm 0 std =======================================================================
[0]	cv-rmse:0.822848
Will train until cv-rmse hasn't improved in 100 rounds.
[100]	cv-rmse:0.822134
Stopping. Best iteration:
[41]	cv-rmse:0.821062

stcktm 1 id =======================================================================
[0]	cv-rmse:2.78552
Will train until cv-rmse hasn't improved in 100 rounds.
[100]	cv-rmse:2.43562
[200]	cv-rmse:2.43321
Stopping. Best iteration:
[198]	cv-rmse:2.4331

stcktm 1 scl =====================================

In [16]:
comdir = top_dir + 'data/progress/committee/'

In [ ]:
bases = [Yx__linfp, Yx__linfp - Y_wmean, dh.Day * 0.0, Y_wmean]
cn__ = []
Yx__ = {}
for packname, ibase, transname in itertools.product(
        ['simple', 'goeasy'], range(len(bases)), ['id', 'scl', 'std']):
    print(packname, ibase, transname, '=======================================================================', flush=True)
    trans = getattr(bt, transname)
    filename = 'split(tc-lb)|base({})*{}{{{}}}'.format(['linFP', 'linFP`1', 0, 1][ibase], transname, packname)
    np.load(thepath + 'ntree/' + filename + '.ntree.npy', bstcv.best_ntree_limit)
    codename = '{}*{}{{{}}}'.format(['linFP', 'linFP`$', '0', '$'][ibase], transname, packname)
    cn__.append(codename)
    Yx__[codename] = np.load(thepath + 'tclb/' + filename + '.Y.npy')

In [ ]:
_nameval = lambda s: ('goeasy' in s) * 1. + ('goeasy' not in s and 'simple' not in s) * .3 + ('simple' in s) * .2
P__ = {cn: (1 if 'linFP' in cn else 1) * _nameval(cn) + (1 / _nameval(cn)) * 0 for cn in cn__}

In [ ]:
Yx__naivecom = sum(Yx__[cn] * P__[cn] for cn in cn__) / sum(P__.values())
np.save(comdir + 'naivecom.npy', Yx__naivecom)

In [ ]:
from scipy.optimize import nnls
Yx_values = np.stack([Yx__[cn] for cn in cn__], axis=1)
beta, _ = nnls(Yx_values[s.tc] * dh.w[s.tc][:, np.newaxis] ** .5, dh.y[s.tc] * dh.w[s.tc] ** .5)
Yx__nnlscom = Yx_values @ beta
np.save(comdir + 'nnlscom.npy', Yx__nnlscom)

In [ ]:
cov_Yx = np.cov(Yx_values, rowvar=False, aweights=dh.w)

In [53]:

s = splitf(dh, 2, ho=40)
X = p.goeasy
Y = dh.y
WY = dh.w
trans = bt.scl

In [54]:
T, WT = trans.to_T(Y, WY)
dmat = make_dmat(X, T, WT, s, include='tr cv aw')

In [55]:
boost_hp = dict(objective='reg:linear', tree_method='hist', booster='gbtree', base_score=0,
                max_depth=3, gamma=50, min_child_weight=1000, eta=.1, seed=0)
train_hp = dict(num_boost_round=10000, early_stopping_rounds=50, evals=[[dmat.tr, 'tr'], [dmat.cv, 'cv']])
#mybst_hp = dict(progress_wait=5, sample_entropy=1, sample_repeat=4, verbose=True)
bst = xgb.train(boost_hp, dmat.tr, **train_hp)

[0]	tr-rmse:1.00174	cv-rmse:0.945263
Multiple eval metrics have been passed: 'cv-rmse' will be used for early stopping.

Will train until cv-rmse hasn't improved in 50 rounds.
[1]	tr-rmse:0.976877	cv-rmse:0.921863
[2]	tr-rmse:0.955664	cv-rmse:0.902526
[3]	tr-rmse:0.937507	cv-rmse:0.88577
[4]	tr-rmse:0.921207	cv-rmse:0.870875
[5]	tr-rmse:0.907328	cv-rmse:0.858481
[6]	tr-rmse:0.895017	cv-rmse:0.847551
[7]	tr-rmse:0.884282	cv-rmse:0.838297
[8]	tr-rmse:0.875065	cv-rmse:0.831011
[9]	tr-rmse:0.866577	cv-rmse:0.824491
[10]	tr-rmse:0.859237	cv-rmse:0.818998
[11]	tr-rmse:0.852498	cv-rmse:0.814127
[12]	tr-rmse:0.846574	cv-rmse:0.809207
[13]	tr-rmse:0.840499	cv-rmse:0.805225
[14]	tr-rmse:0.835672	cv-rmse:0.801812
[15]	tr-rmse:0.831495	cv-rmse:0.798634
[16]	tr-rmse:0.827377	cv-rmse:0.795803
[17]	tr-rmse:0.823629	cv-rmse:0.793449
[18]	tr-rmse:0.820315	cv-rmse:0.791124
[19]	tr-rmse:0.817387	cv-rmse:0.789141
[20]	tr-rmse:0.81471	cv-rmse:0.787441
[21]	tr-rmse:0.81226	cv-rmse:0.785495
[22]	tr-rmse:0.80

[206]	tr-rmse:0.724324	cv-rmse:0.761424
[207]	tr-rmse:0.724143	cv-rmse:0.761344
[208]	tr-rmse:0.723879	cv-rmse:0.761366
[209]	tr-rmse:0.723721	cv-rmse:0.761397
[210]	tr-rmse:0.723463	cv-rmse:0.761382
[211]	tr-rmse:0.723181	cv-rmse:0.761395
[212]	tr-rmse:0.722907	cv-rmse:0.761301
[213]	tr-rmse:0.72265	cv-rmse:0.761235
[214]	tr-rmse:0.722452	cv-rmse:0.761257
[215]	tr-rmse:0.722211	cv-rmse:0.761357
[216]	tr-rmse:0.721946	cv-rmse:0.761437
[217]	tr-rmse:0.721845	cv-rmse:0.761366
[218]	tr-rmse:0.721677	cv-rmse:0.761416
[219]	tr-rmse:0.721516	cv-rmse:0.761509
[220]	tr-rmse:0.72124	cv-rmse:0.761543
[221]	tr-rmse:0.720967	cv-rmse:0.761538
[222]	tr-rmse:0.720735	cv-rmse:0.761527
[223]	tr-rmse:0.720507	cv-rmse:0.761537
[224]	tr-rmse:0.720352	cv-rmse:0.76185
[225]	tr-rmse:0.720127	cv-rmse:0.761818
[226]	tr-rmse:0.719865	cv-rmse:0.761893
[227]	tr-rmse:0.71963	cv-rmse:0.761751
[228]	tr-rmse:0.719416	cv-rmse:0.761732
[229]	tr-rmse:0.719251	cv-rmse:0.761707
[230]	tr-rmse:0.719052	cv-rmse:0.76163
[231]

In [56]:
bst.best_ntree_limit #-40

307

In [25]:
bst.best_ntree_limit #1/3

597

In [20]:
bst.best_ntree_limit #1/2

411

In [ ]:
# bt.scl
[28]	tr-rmse:0.749604	cv-rmse:0.785497

In [24]:
Tx = bst.predict(dmat.aw)
Yx = trans.to_Y(Tx)

In [63]:
thing = np.load(thepath + 'tclb/' + 'split(tc-lb)|base(1)*std{goeasy}' + '.Y.npy')

In [68]:
SE = s.tc
((Y[SE] - thing[SE]) ** 2 * WY[SE]).sum() / (Y[SE] ** 2 * WY[SE]).sum()

0.48412281272167551

In [15]:
SE = s.cv
((Y[SE] - (Yx_std[SE] + Yx_id[SE] + Yx[SE]) / 3) ** 2 * WY[SE]).sum() / (Y[SE] ** 2 * WY[SE]).sum()

0.61081138224311582

In [21]:
((Y - Yx) ** 2 * WY).sum() / (Y ** 2 * WY).sum()

0.54593558741401083

In [37]:
s.repr

'split(a=f,f=2,r=1,h=0,v=0)'

In [12]:
#np.save(top_dir + 'data/progress/xgboost/halfass/std/' + s.repr + '|0*std.npy', Tx)
Yx_load = np.load(top_dir + 'data/progress/xgboost/halfass/id/' + s.repr + '|0*std.npy')

In [19]:
np.save(top_dir + 'data/progress/xgboost/halfass/scl/' + s.repr + '|0*scl.npy', Tx)
np.save(top_dir + 'data/progress/xgboost/halfass/id/' + s.repr + '|0*scl.npy', Yx)

In [26]:
submit(Yx, 'Ystd')

### hyper

In [ ]:
# bt.id
boost_hp = dict(objective='reg:linear', tree_method='exact', booster='gbtree', base_score=0,
                max_depth=3, gamma=50, min_child_weight=1000, eta=.5, seed=0)
train_hp = dict(num_boost_round=10000, early_stopping_rounds=100, evals=[[dmat.tr, 'tr'], [dmat.cv, 'cv']])
#mybst_hp = dict(progress_wait=5, sample_entropy=1, sample_repeat=4, verbose=True)
bst = xgb.train(boost_hp, dmat.tr, **train_hp)

### testing legacy myboost :(

In [524]:
boost_hp = dict(objective='reg:linear', tree_method='exact', booster='gbtree', base_score=0,
                max_depth=3, gamma=50, min_child_weight=1000, eta=.5, seed=0)
train_hp = dict(num_boost_round=10000, early_stopping_rounds=100, evals=[[dmat.tr, 'tr'], [dmat.cv, 'cv'], [dmat.ho, 'ho']])
#mybst_hp = dict(progress_wait=5, sample_entropy=1, sample_repeat=4, verbose=True)
bst1 = xgb.train(boost_hp, dmat.tr, **train_hp)

[0]	tr-rmse:3.06125	cv-rmse:2.96365	ho-rmse:3.00835
Multiple eval metrics have been passed: 'ho-rmse' will be used for early stopping.

Will train until ho-rmse hasn't improved in 100 rounds.
[1]	tr-rmse:2.90357	cv-rmse:2.81808	ho-rmse:2.86903
[2]	tr-rmse:2.8236	cv-rmse:2.75076	ho-rmse:2.80982
[3]	tr-rmse:2.77597	cv-rmse:2.71523	ho-rmse:2.77632
[4]	tr-rmse:2.74952	cv-rmse:2.69902	ho-rmse:2.75804
[5]	tr-rmse:2.7257	cv-rmse:2.68015	ho-rmse:2.73597
[6]	tr-rmse:2.70693	cv-rmse:2.66608	ho-rmse:2.72279
[7]	tr-rmse:2.69545	cv-rmse:2.65858	ho-rmse:2.71426
[8]	tr-rmse:2.67824	cv-rmse:2.646	ho-rmse:2.70145
[9]	tr-rmse:2.66684	cv-rmse:2.63577	ho-rmse:2.69012
[10]	tr-rmse:2.65981	cv-rmse:2.63044	ho-rmse:2.68806
[11]	tr-rmse:2.65424	cv-rmse:2.62784	ho-rmse:2.68326
[12]	tr-rmse:2.64682	cv-rmse:2.62414	ho-rmse:2.67779
[13]	tr-rmse:2.64211	cv-rmse:2.62104	ho-rmse:2.67362
[14]	tr-rmse:2.63569	cv-rmse:2.61675	ho-rmse:2.67102
[15]	tr-rmse:2.6279	cv-rmse:2.61389	ho-rmse:2.66808
[16]	tr-rmse:2.62188	cv-rms

[153]	tr-rmse:2.39563	cv-rmse:2.58603	ho-rmse:2.63702
[154]	tr-rmse:2.39484	cv-rmse:2.58615	ho-rmse:2.63717
[155]	tr-rmse:2.3941	cv-rmse:2.58535	ho-rmse:2.63646
[156]	tr-rmse:2.39359	cv-rmse:2.58487	ho-rmse:2.63652
[157]	tr-rmse:2.39315	cv-rmse:2.58484	ho-rmse:2.63634
[158]	tr-rmse:2.39187	cv-rmse:2.58429	ho-rmse:2.63589
[159]	tr-rmse:2.3914	cv-rmse:2.58493	ho-rmse:2.63631
[160]	tr-rmse:2.38987	cv-rmse:2.58467	ho-rmse:2.63594
[161]	tr-rmse:2.38803	cv-rmse:2.58588	ho-rmse:2.63703
[162]	tr-rmse:2.38683	cv-rmse:2.58665	ho-rmse:2.63727
[163]	tr-rmse:2.38639	cv-rmse:2.58688	ho-rmse:2.63731
[164]	tr-rmse:2.38602	cv-rmse:2.58702	ho-rmse:2.63769
[165]	tr-rmse:2.38455	cv-rmse:2.58585	ho-rmse:2.63782
[166]	tr-rmse:2.38342	cv-rmse:2.58615	ho-rmse:2.63811
[167]	tr-rmse:2.38299	cv-rmse:2.58669	ho-rmse:2.63766
[168]	tr-rmse:2.38264	cv-rmse:2.58636	ho-rmse:2.63762
[169]	tr-rmse:2.38064	cv-rmse:2.58534	ho-rmse:2.6368
[170]	tr-rmse:2.37967	cv-rmse:2.58564	ho-rmse:2.63629
[171]	tr-rmse:2.3783	cv-rmse:2.

In [530]:
boost_hp = dict(objective='reg:linear', tree_method='exact', booster='gbtree', base_score=0,
                max_depth=3, gamma=50, min_child_weight=1000, eta=.5, seed=0)
#train_hp = dict(num_boost_round=100, early_stopping_rounds=5, evals=[[dtst.tr, 'TR'], [dtst.cv, 'CV'], [dtst.ho, 'HO']])
mybst_hp = dict(progress_wait=100, sample_entropy=.5, sample_repeat=8, verbose=True)
bst2 = myboost(dmat, boost_hp, **mybst_hp)

std(tr:0.016095,cv:0.013134,ho:0.010076) min(tr:3.04857,cv:2.944856,ho:2.995571) max(tr:3.10363,cv:2.991976,ho:3.031176)
[0]	tr-rmse:3.048570	cv-rmse:2.944856	ho-rmse:2.995571

std(tr:0.003098,cv:0.006638,ho:0.008201) min(tr:2.899248,cv:2.798263,ho:2.860801) max(tr:2.907204,cv:2.814655,ho:2.881927)
[1]	tr-rmse:2.899937	cv-rmse:2.798263	ho-rmse:2.860801

std(tr:0.007405,cv:0.007062,ho:0.0066) min(tr:2.81242,cv:2.728332,ho:2.795226) max(tr:2.827069,cv:2.744923,ho:2.810519)
[2]	tr-rmse:2.812521	cv-rmse:2.728332	ho-rmse:2.795407

std(tr:0.005153,cv:0.003811,ho:0.004449) min(tr:2.762366,cv:2.689974,ho:2.76054) max(tr:2.77423,cv:2.698817,ho:2.770696)
[3]	tr-rmse:2.762366	cv-rmse:2.689974	ho-rmse:2.760540

std(tr:0.002103,cv:0.002441,ho:0.002859) min(tr:2.734241,cv:2.665343,ho:2.737561) max(tr:2.739201,cv:2.67076,ho:2.744195)
[4]	tr-rmse:2.734256	cv-rmse:2.665343	ho-rmse:2.737561

std(tr:0.004587,cv:0.003826,ho:0.004226) min(tr:2.704177,cv:2.645783,ho:2.712787) max(tr:2.719249,cv:2.657372,ho:

std(tr:7.7e-05,cv:0.000458,ho:0.000362) min(tr:2.56825,cv:2.574077,ho:2.653727) max(tr:2.568429,cv:2.575201,ho:2.65487)
[46]	tr-rmse:2.568429	cv-rmse:2.574077	ho-rmse:2.654337

std(tr:0.000521,cv:0.000449,ho:0.000358) min(tr:2.567165,cv:2.574987,ho:2.654047) max(tr:2.568377,cv:2.57611,ho:2.655306)
[47]	tr-rmse:2.568250	cv-rmse:2.574987	ho-rmse:2.654515

std(tr:0.000648,cv:0.000801,ho:0.000217) min(tr:2.567117,cv:2.574077,ho:2.654042) max(tr:2.56851,cv:2.576116,ho:2.654756)
[48]	tr-rmse:2.568429	cv-rmse:2.574077	ho-rmse:2.654337

std(tr:0.000407,cv:0.000388,ho:0.000237) min(tr:2.567165,cv:2.574914,ho:2.654047) max(tr:2.568377,cv:2.57611,ho:2.654893)
[49]	tr-rmse:2.568276	cv-rmse:2.574914	ho-rmse:2.654471

std(tr:0.00067,cv:0.000471,ho:0.000196) min(tr:2.567004,cv:2.574976,ho:2.65425) max(tr:2.568591,cv:2.576095,ho:2.654827)
[50]	tr-rmse:2.568591	cv-rmse:2.574976	ho-rmse:2.654366

std(tr:0.000545,cv:0.000404,ho:0.003585) min(tr:2.567193,cv:2.57456,ho:2.644404) max(tr:2.568558,cv:2.575727

std(tr:6.7e-05,cv:0.000294,ho:0.000177) min(tr:2.554508,cv:2.571004,ho:2.645569) max(tr:2.55473,cv:2.571909,ho:2.646144)
[92]	tr-rmse:2.554730	cv-rmse:2.571004	ho-rmse:2.645846

std(tr:0.000522,cv:0.000491,ho:0.000255) min(tr:2.554542,cv:2.570905,ho:2.645413) max(tr:2.555658,cv:2.572205,ho:2.646144)
[93]	tr-rmse:2.555658	cv-rmse:2.570905	ho-rmse:2.645891

std(tr:0.000733,cv:0.000697,ho:0.000307) min(tr:2.553581,cv:2.569818,ho:2.645328) max(tr:2.555664,cv:2.571632,ho:2.646273)
[94]	tr-rmse:2.553581	cv-rmse:2.569818	ho-rmse:2.645563

std(tr:0.000375,cv:0.000705,ho:0.000142) min(tr:2.554641,cv:2.569816,ho:2.645848) max(tr:2.555739,cv:2.57197,ho:2.646273)
[95]	tr-rmse:2.555739	cv-rmse:2.569816	ho-rmse:2.646252

std(tr:0.000754,cv:0.000763,ho:0.000357) min(tr:2.553558,cv:2.569553,ho:2.645327) max(tr:2.555756,cv:2.57197,ho:2.646322)
[96]	tr-rmse:2.553558	cv-rmse:2.569553	ho-rmse:2.645468

std(tr:0.000639,cv:0.0012,ho:0.000618) min(tr:2.549337,cv:2.568247,ho:2.643951) max(tr:2.550953,cv:2.571

std(tr:0.000413,cv:0.000127,ho:0.000241) min(tr:2.531395,cv:2.56604,ho:2.639949) max(tr:2.532806,cv:2.566422,ho:2.640599)
[138]	tr-rmse:2.531395	cv-rmse:2.566040	ho-rmse:2.639949

std(tr:0.000197,cv:0.000322,ho:0.000286) min(tr:2.532256,cv:2.565495,ho:2.639776) max(tr:2.532761,cv:2.566457,ho:2.640599)
[139]	tr-rmse:2.532761	cv-rmse:2.565495	ho-rmse:2.639776

std(tr:0.000216,cv:0.000144,ho:0.000254) min(tr:2.532256,cv:2.566077,ho:2.639895) max(tr:2.532921,cv:2.566499,ho:2.640625)
[140]	tr-rmse:2.532921	cv-rmse:2.566077	ho-rmse:2.639895

std(tr:0.000366,cv:0.000167,ho:0.000387) min(tr:2.531727,cv:2.566085,ho:2.640098) max(tr:2.532912,cv:2.566543,ho:2.641305)
[141]	tr-rmse:2.532912	cv-rmse:2.566085	ho-rmse:2.640263

std(tr:0.000138,cv:0.000245,ho:0.000135) min(tr:2.532256,cv:2.565687,ho:2.640313) max(tr:2.532573,cv:2.566422,ho:2.640708)
[142]	tr-rmse:2.532573	cv-rmse:2.565687	ho-rmse:2.640363

std(tr:0.000192,cv:0.000173,ho:0.000357) min(tr:2.532256,cv:2.566108,ho:2.639641) max(tr:2.53273

std(tr:0.000179,cv:0.000159,ho:0.000157) min(tr:2.532256,cv:2.566307,ho:2.640236) max(tr:2.532873,cv:2.566785,ho:2.640599)
[184]	tr-rmse:2.532533	cv-rmse:2.566307	ho-rmse:2.640313

std(tr:0.0003,cv:0.000129,ho:0.000316) min(tr:2.531597,cv:2.566141,ho:2.639666) max(tr:2.532482,cv:2.566543,ho:2.640599)
[185]	tr-rmse:2.531597	cv-rmse:2.566141	ho-rmse:2.639666

std(tr:0.000281,cv:0.000279,ho:0.000284) min(tr:2.531748,cv:2.565658,ho:2.63977) max(tr:2.532707,cv:2.566543,ho:2.640599)
[186]	tr-rmse:2.531748	cv-rmse:2.565658	ho-rmse:2.639770

std(tr:0.000342,cv:0.000165,ho:0.000491) min(tr:2.531724,cv:2.566085,ho:2.640263) max(tr:2.532912,cv:2.566641,ho:2.641828)
[187]	tr-rmse:2.532912	cv-rmse:2.566085	ho-rmse:2.640263

std(tr:0.000375,cv:0.000314,ho:0.000786) min(tr:2.531661,cv:2.565757,ho:2.639786) max(tr:2.532833,cv:2.56681,ho:2.642265)
[188]	tr-rmse:2.532614	cv-rmse:2.565757	ho-rmse:2.639817

std(tr:7.9e-05,cv:4.7e-05,ho:0.000158) min(tr:2.532256,cv:2.566422,ho:2.640245) max(tr:2.532482,cv:

std(tr:0.000488,cv:0.000377,ho:0.000274) min(tr:2.530048,cv:2.565894,ho:2.639868) max(tr:2.531466,cv:2.566842,ho:2.640655)
[230]	tr-rmse:2.530048	cv-rmse:2.565894	ho-rmse:2.640414

std(tr:0.000338,cv:0.000284,ho:0.000502) min(tr:2.530344,cv:2.565854,ho:2.640345) max(tr:2.531446,cv:2.566897,ho:2.641972)
[231]	tr-rmse:2.531396	cv-rmse:2.565854	ho-rmse:2.640345

std(tr:0.000339,cv:0.000271,ho:0.000258) min(tr:2.530466,cv:2.565854,ho:2.640345) max(tr:2.5315,cv:2.566723,ho:2.641193)
[232]	tr-rmse:2.531396	cv-rmse:2.565854	ho-rmse:2.640345

std(tr:0.00048,cv:0.000178,ho:0.000287) min(tr:2.530024,cv:2.566159,ho:2.640234) max(tr:2.5315,cv:2.566723,ho:2.641256)
[233]	tr-rmse:2.530024	cv-rmse:2.566159	ho-rmse:2.641256

std(tr:0.000166,cv:0.0003,ho:0.000224) min(tr:2.531115,cv:2.565976,ho:2.640063) max(tr:2.531521,cv:2.566723,ho:2.640655)
[234]	tr-rmse:2.531264	cv-rmse:2.565976	ho-rmse:2.640063

std(tr:0.000408,cv:0.000254,ho:0.000804) min(tr:2.530447,cv:2.566248,ho:2.640234) max(tr:2.531482,cv:2

std(tr:0.000297,cv:0.000179,ho:0.000299) min(tr:2.530369,cv:2.566085,ho:2.640573) max(tr:2.531311,cv:2.566619,ho:2.64147)
[276]	tr-rmse:2.531311	cv-rmse:2.566085	ho-rmse:2.640573

std(tr:0.000496,cv:0.000368,ho:0.000264) min(tr:2.530048,cv:2.565542,ho:2.640003) max(tr:2.531466,cv:2.566723,ho:2.640914)
[277]	tr-rmse:2.530380	cv-rmse:2.565542	ho-rmse:2.640003

std(tr:0.00016,cv:0.000194,ho:0.000152) min(tr:2.531115,cv:2.566085,ho:2.640234) max(tr:2.531482,cv:2.566723,ho:2.640714)
[278]	tr-rmse:2.531311	cv-rmse:2.566085	ho-rmse:2.640573

std(tr:0.000299,cv:0.000263,ho:0.000263) min(tr:2.530463,cv:2.565976,ho:2.640063) max(tr:2.531466,cv:2.566723,ho:2.640655)
[279]	tr-rmse:2.531264	cv-rmse:2.565976	ho-rmse:2.640063

std(tr:0.000171,cv:0.000333,ho:0.000141) min(tr:2.531115,cv:2.565854,ho:2.640345) max(tr:2.531466,cv:2.566723,ho:2.640655)
[280]	tr-rmse:2.531396	cv-rmse:2.565854	ho-rmse:2.640345

std(tr:0.000488,cv:0.000296,ho:0.000392) min(tr:2.530004,cv:2.565697,ho:2.640363) max(tr:2.531466

KeyboardInterrupt: 

# testing

In [20]:
model = None
boost_hp = dict(
    objective = 'reg:linear',
    booster = 'gbtree',
    max_depth = 3, #12,
    subsample = 1,
    colsample_bytree = 1,
    colsample_bylevel = 1,
    tree_method = 'hist',
    gamma = 50.00003,
    min_child_weight = 1000,
    eta = 0.1,
    seed = 0
)
cv_hp = dict(
    num_boost_round = 300,
    early_stopping_rounds = 30,
    nfold = 10,
    stratified = True,
    seed = 0
)
learn_hp = dict(
    num_boost_round = 1,
    early_stopping_rounds = 100,
    evals = [[dmat.tr, 'Train'], [dmat.cv, 'CV']]
)
bst = xgb.train(boost_hp, dmat.tr, **learn_hp)

[0]	Train-rmse:3.30618	CV-rmse:3.19202
Multiple eval metrics have been passed: 'CV-rmse' will be used for early stopping.

Will train until CV-rmse hasn't improved in 100 rounds.


In [3]:
Xvalues = np.array([
    [0., 1, 2, 50],
    [1, 1, 4, 20],
    [0, 2, 2, 20],
    [0, 3, 2, 10],
    [0, 4, -100, 4],
    [0, 5, np.nan, 50],
    [0, 1, 50, 50],      #6
    [1, 2, 5, 10],
    [0, 2, np.nan, 20],
    [0, 3, 2, 5],
    [0, 1, 10, 4],
    [0, 5, np.nan, 30],
    [0, 5, 2, 10],       #7
    [0, 1, 0, 20],
    [0, 2, 2, 10],
    [0, 4, 2, 10],
    [1, 4, -50, 40],
    [0, 5, 0, 30],
])
Y = pd.Series(np.array([5., 10, 6, 6, 15, -5,    20, 1, 10, 7, 7, 12,    17, 2, 5, 8, 6, 8]))
W = pd.Series(np.array([1., 1, 1, 1, 1, 1,    1, 1, 1, 1, 1, 1,    1, 1, 1, 1, 1, 1]))
ftst = 'Special Alpha Noisy Agg'.split()
X = pd.DataFrame(Xvalues, columns=ftst)
s = O(tr=slice(0, 6), cv=slice(6, 12), ho=slice(12, 18))
dtst = make_dmat(X, Y, W, s)
#dmat save binary

In [30]:
def get_dmat(S, T, W, s, name, include, new=False):
    include = include.split() if isinstance(include, str) else include
    dmat = O()
    from path import Path
    path = Path(top_dir + 'data/progress/dmat/' + s.repr() + '|' + name)
    if path.is_dir():
        assert not new, '`new` must be False when folder exists'
        dmat = O()
        for se in include:
            dmat[se] = xgb.DMatrix(str(path + '.' + se))
    else:
        assert new, '`new` must be True when folder does not exist'
        dmat = make_dmat(S, T, W, s, include=include)
        path.mkdir()
        for se in include:
            dmat[se].save_binary(str(path + '.' + se))
    return dmat

In [390]:
boost_hp = dict(objective='reg:linear', tree_method='exact', booster='gbtree', base_score=0,
                max_depth=2, colsample_bytree=.2, seed=0)
learn_hp = dict(num_boost_round=100, early_stopping_rounds=5, evals=[[dtst.tr, 'TR'], [dtst.cv, 'CV'], [dtst.ho, 'HO']])
bst0 = xgb.train(boost_hp, dtst.tr, **learn_hp)
bst0.set_param(boost_hp)

[0]	TR-rmse:7.58006	CV-rmse:9.80914	HO-rmse:7.65072
Multiple eval metrics have been passed: 'HO-rmse' will be used for early stopping.

Will train until HO-rmse hasn't improved in 5 rounds.
[1]	TR-rmse:6.932	CV-rmse:8.88602	HO-rmse:6.7524
[2]	TR-rmse:6.54696	CV-rmse:8.24244	HO-rmse:6.14653
[3]	TR-rmse:6.31349	CV-rmse:7.97129	HO-rmse:5.84308
[4]	TR-rmse:5.3583	CV-rmse:8.93732	HO-rmse:5.49028
[5]	TR-rmse:4.57386	CV-rmse:8.9271	HO-rmse:6.02277
[6]	TR-rmse:3.9298	CV-rmse:8.9715	HO-rmse:6.57722
[7]	TR-rmse:3.83046	CV-rmse:8.97181	HO-rmse:6.59058
[8]	TR-rmse:3.75787	CV-rmse:8.98354	HO-rmse:6.61288
[9]	TR-rmse:3.2349	CV-rmse:9.59871	HO-rmse:6.65334
Stopping. Best iteration:
[4]	TR-rmse:5.3583	CV-rmse:8.93732	HO-rmse:5.49028



In [331]:
bst = bst1.copy() #xgb.Booster(boost_hp, [dtst.tr])
bst.set_param(boost_hp)
i = 0
bst.set_param('seed', i)

In [334]:
bst.update(dtst.tr, i)
i += 1
bst.set_param('seed', i)

In [319]:
bst1 = bst.copy()

In [386]:
line = bst.eval_set([[dtst.tr, 'tr'], [dtst.cv, 'cv'], [dtst.ho, 'ho']], 0)
print(repr(line))

'[0]\ttr-rmse:4.076989\tcv-rmse:10.518982\tho-rmse:5.435194'


In [398]:
err_ = [xgb_eval_parse(line, 3) for i in range(4)]
err_

[<>(_orig='[0]\ttr-rmse:4.076989\tcv-rmse:10.518982\tho-rmse:5.435194',
  cv=10.518982,
  ho=5.435194,
  tr=4.076989),
 <>(_orig='[0]\ttr-rmse:4.076989\tcv-rmse:10.518982\tho-rmse:5.435194',
  cv=10.518982,
  ho=5.435194,
  tr=4.076989),
 <>(_orig='[0]\ttr-rmse:4.076989\tcv-rmse:10.518982\tho-rmse:5.435194',
  cv=10.518982,
  ho=5.435194,
  tr=4.076989),
 <>(_orig='[0]\ttr-rmse:4.076989\tcv-rmse:10.518982\tho-rmse:5.435194',
  cv=10.518982,
  ho=5.435194,
  tr=4.076989)]

In [501]:
def myboost(dmat, hp, ho=True, *, progress_wait, sample_entropy, sample_repeat, boost_limit=1000000000,
            print=print, verbose=False):
    # basic init
    s_fields = ['tr', 'cv'] + (['ho'] if ho else [])
    from collections import namedtuple
    MyError = namedtuple('MyError', s_fields)
    hp = O(**hp)
    
    assert not any(['subsample' in hp,
                    'colsample_bytree' in hp,
                    'colsample_bylevel' in hp]), 'Subsampling is handled by `myboost` internally'
    hp.seed = hp.seed - 1 if 'seed' in hp else -1 # `seed += 1` happens first in loops
    hp.base_score = 0
    max_depth, hp.max_depth = hp.max_depth, 0
    evals = [[dmat[se], se] for se in s_fields]
    bst0 = xgb.train(O.py(hp), dmat.tr, num_boost_round=0, evals=evals)
    hp.max_depth = max_depth
    bst0.set_param(O.py(hp))
    # myboost params init
    assert not(sample_entropy == 0 and sample_repeat > 1), '`sample_repeat` must > 1 if `sample_entropy == 0`'
    hp.colsample_bylevel = np.exp(-sample_entropy)
    # boost iterations
    i, i_best, i_progress = -1, -1, -1
    bst1 = bst0 # 1 here means "best so far", not like uh 1
    err1 = xgb_eval(bst0, evals, i) # same here
    for i in range(boost_limit):
        bst_, err_ = [], []
        for j in range(sample_repeat):
            hp.seed += 1
            bst = bst1.copy()
            bst.set_param(O.py(hp))
            bst.update(dmat.tr, i)
            err = xgb_eval(bst, evals, i)
            bst_.append(bst)
            err_.append(err)
        j = min(range(sample_repeat), key=lambda j: err_[j].cv)
        bst, err = bst_[j], err_[j]
        if verbose:
            _myboost_print_err_info(err_, ho=ho, print=print)
        print(err._orig)
        if verbose:
            print()
        if err.cv < err1.cv:
            bst1, err1 = bst, err
            i_progress = i
        elif i - i_progress >= progress_wait:
            break
    print('No progress. Best iteration:')
    print(err1._orig, flush=True)
    return bst1

In [515]:
def _myboost_print_err_info(err_, ho=True, print=print):
    ho=True
    fields = ['tr', 'cv'] + (['ho'] if ho else [])
    err = pd.DataFrame({k: [err_[i][k] for i in range(len(err_))] for k in fields})
    err = err[fields]
    print('std' + '(' + ','.join('{}:{}'.format(f, v) for f, v in zip(fields, np.round(err.std() * 1e6) / 1e6)) + ')',
          'min' + '(' + ','.join('{}:{}'.format(f, v) for f, v in zip(fields, err.min())) + ')',
          'max' + '(' + ','.join('{}:{}'.format(f, v) for f, v in zip(fields, err.max())) + ')')

In [442]:
def xgb_eval(bst, evals, i):
    n = len(evals)
    line = bst.eval_set(evals, i)
    import re
    m = re.match(r'\[\-?\d+\]' + r'\t(\w+)\-[^\:]+\:(\S+)' * n, line)
    assert m and len(m.groups()) == n * 2, (line, m)
    fields, values = m.groups()[::2], tuple(float(x) for x in m.groups()[1::2])
    return O(**{f: v for f, v in zip(fields, values)}, _orig=line)

In [517]:
boost_hp = dict(objective='reg:linear', tree_method='exact', booster='gbtree', base_score=0,
                max_depth=2, seed=0)
train_hp = dict(num_boost_round=100, early_stopping_rounds=5, evals=[[dtst.tr, 'TR'], [dtst.cv, 'CV'], [dtst.ho, 'HO']])
mybst_hp = dict(progress_wait=5, sample_entropy=1, sample_repeat=8, verbose=True)
bst = myboost(dtst, boost_hp, **mybst_hp)

std(tr:0.333944,cv:0.860651,ho:0.188335) min(tr:6.900634,cv:9.780291,ho:7.248678) max(tr:7.580062,cv:11.503405,ho:7.650722)
[0]	tr-rmse:6.934919	cv-rmse:9.780291	ho-rmse:7.512262

std(tr:0.277377,cv:0.449033,ho:0.396612) min(tr:5.654825,cv:8.886453,ho:6.310345) max(tr:6.184599,cv:10.184079,ho:7.385381)
[1]	tr-rmse:6.184599	cv-rmse:8.886453	ho-rmse:6.600405

std(tr:0.004709,cv:0.514374,ho:0.480677) min(tr:5.172213,cv:8.283962,ho:5.72908) max(tr:5.185532,cv:9.4614,ho:6.736963)
[2]	tr-rmse:5.185532	cv-rmse:8.283962	ho-rmse:6.013216

std(tr:0.169692,cv:0.293165,ho:0.192166) min(tr:4.393541,cv:7.889721,ho:5.482253) max(tr:4.882342,cv:8.766361,ho:6.091563)
[3]	tr-rmse:4.420090	cv-rmse:7.889721	ho-rmse:5.482253

std(tr:0.14386,cv:0.294136,ho:0.281296) min(tr:3.771114,cv:7.599853,ho:4.772295) max(tr:4.18509,cv:8.319516,ho:5.653121)
[4]	tr-rmse:4.185090	cv-rmse:7.599853	ho-rmse:5.216857

std(tr:0.059868,cv:0.206638,ho:0.171148) min(tr:3.597699,cv:7.538013,ho:4.945333) max(tr:3.720961,cv:8.07188